In [2]:
import ujson
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score

FEATURE_POSES = ["NC", "NQ"]
INPUT_FILE_NAME = "../data/speeches/nmh_speeches.ma.txt"
MA_KEY = "body_ma"
LABEL_KEY = "field"
EXCL_LABEL = "환경"


def read_documents_with_labels(input_file_name):
    documents = []
    labels = []

    with open(input_file_name, "r", encoding="utf-8") as input_file:
        for line in input_file:
            morphs = []
            json_obj = ujson.loads(line)
            label = json_obj[LABEL_KEY]
            
            if label == EXCL_LABEL:
                continue
                
            labels.append(label)

            for sent_anal in json_obj[MA_KEY]:
                for morph_lex, morph_cat in sent_anal:
                    if morph_cat not in FEATURE_POSES:
                        continue

                    morphs.append(morph_lex)

            document = " ".join(morphs)
            documents.append(document)

    # scikit의 다양한 기능을 이용하기 위해서
    # 파이썬 리스트를 numpy 모듈의 배열(array)로 바꾼다.
    documents = np.asarray(documents)
    labels = np.asarray(labels)
    
    return documents, labels


def build_pipeline():
    vectorizer = TfidfVectorizer(tokenizer=str.split)
    clf = MultinomialNB()
    pipeline = Pipeline([
        ('vect', vectorizer),
        ('clf', clf),
    ])
    
    return pipeline


def main():
    input_file_name = "../data/speeches/nmh_speeches.ma.txt"
    documents, labels = read_documents_with_labels(input_file_name)
    train_documents, test_documents, train_labels, test_labels = \
            train_test_split(documents, labels)
    pipeline = build_pipeline()

    parameters = {
        "vect__max_features": (5000, None),
        "vect__ngram_range": ((1, 1), (1, 2)),
        "vect__use_idf": (True, False),
        "vect__smooth_idf": (True, False),
        "vect__sublinear_tf": (True, False),
        "vect__norm": ("l1", "l2", None),
        "clf__loss": ("hinge", "log"),
        "clf__penalty": ("l1","l2","none"),
        "clf__alpha": (0.0001, 0.05, 0.1),
    }

    grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, 
                               scoring="accuracy", cv=3)
    grid_search.fit(train_documents, train_labels)
    best_parameters = grid_search.best_estimator_.get_params()
    
    pred_labels = grid_search.predict(test_documents)

    print("Best score: {}".format(grid_search.best_score_))

    print("Best parameter set:")

    for param_name in parameters:
        print("\t{}: {}".format(param_name, best_parameters[param_name]))

    print("Accurary: {}".format(accuracy_score(test_labels, pred_labels)))


# 실행
main()

Fitting 3 folds for each of 1728 candidates, totalling 5184 fits


JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\ProgramData\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel.__main__', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel.__main__', loader=<_f...da3\\lib\\site-packages\\ipykernel\\__main__.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\ProgramData\Anaconda3\lib\runpy.py